MARKET BASKET ANALYSIS

In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from mlxtend.frequent_patterns import apriori, association_rules

CNVERTING THE DATASET INTO SINGLE CSV

In [62]:
def read_excel_sheets(xls_path):
    """membaca semua sheet yang ada didalam excel
    dan mengembalikan ke dalam satu tipe dataframe"""
    print(f'Loading {xls_path} into pandas')
    xl = pd.ExcelFile(xls_path)
    df = pd.DataFrame()
    columns = None
    for idx, name in enumerate(xl.sheet_names):
        print(f'Reading sheet #{idx}: {name}')
        sheet = xl.parse(name)
        if idx == 0:
            #menyimpan nama kolom dari sheet pertama yang cocok ke dala append
            columns = sheet.columns
        sheet.columns = columns
        #Asumsi index untuk data yang sudah ada ketika appended
        df = df.append(sheet, ignore_index=True)
    return df

In [44]:
stops = read_excel_sheets("Superstore_Data_One_Region_Per_Sheet.xlsx")
stops.to_csv("Superstore_data.csv", index=False)

Loading Superstore_Data_One_Region_Per_Sheet.xlsx into pandas
Reading sheet #0: West_Superstore
Reading sheet #1: Central_Superstore
Reading sheet #2: East_Superstore
Reading sheet #3: South_Superstore


READING THE NEW DATASET

In [45]:
df = pd.read_csv('Superstore_data.csv')
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,5328,CA-2011-130813,2013-01-06,2013-01-08,Second Class,LS-17230,Lycoris Saunders,Consumer,United States,Los Angeles,...,90049,West,OFF-PA-10002005,Office Supplies,Paper,Xerox 225,19.440,3,0.00,9.3312
1,4938,CA-2011-157147,2013-01-13,2013-01-18,Standard Class,BD-11605,Brian Dahlen,Consumer,United States,San Francisco,...,94109,West,OFF-ST-10000078,Office Supplies,Storage,Tennsco 6- and 18-Compartment Lockers,1325.850,5,0.00,238.6530
2,4939,CA-2011-157147,2013-01-13,2013-01-18,Standard Class,BD-11605,Brian Dahlen,Consumer,United States,San Francisco,...,94109,West,FUR-BO-10003034,Furniture,Bookcases,"O'Sullivan Elevations Bookcase, Cherry Finish",333.999,3,0.15,3.9294
3,4940,CA-2011-157147,2013-01-13,2013-01-18,Standard Class,BD-11605,Brian Dahlen,Consumer,United States,San Francisco,...,94109,West,OFF-AR-10003514,Office Supplies,Art,4009 Highlighters by Sanford,19.900,5,0.00,6.5670
4,5365,CA-2011-123477,2013-01-18,2013-01-21,Second Class,DW-13195,David Wiener,Corporate,United States,Springfield,...,97477,West,OFF-AP-10000692,Office Supplies,Appliances,Fellowes Mighty 8 Compact Surge Protector,64.864,4,0.20,6.4864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,7633,US-2014-158526,2016-12-29,2017-01-01,Second Class,KH-16360,Katherine Hughes,Consumer,United States,Louisville,...,40214,South,FUR-CH-10002602,Furniture,Chairs,DMI Arturo Collection Mission-style Design Woo...,1207.840,8,0.00,314.0384
9990,7634,US-2014-158526,2016-12-29,2017-01-01,Second Class,KH-16360,Katherine Hughes,Consumer,United States,Louisville,...,40214,South,OFF-BI-10002414,Office Supplies,Binders,GBC ProClick Spines for 32-Hole Punch,12.530,1,0.00,5.8891
9991,7635,US-2014-158526,2016-12-29,2017-01-01,Second Class,KH-16360,Katherine Hughes,Consumer,United States,Louisville,...,40214,South,OFF-AR-10003696,Office Supplies,Art,Panasonic KP-350BK Electric Pencil Sharpener w...,34.580,1,0.00,10.0282
9992,7636,US-2014-158526,2016-12-29,2017-01-01,Second Class,KH-16360,Katherine Hughes,Consumer,United States,Louisville,...,40214,South,FUR-CH-10004495,Furniture,Chairs,"Global Leather and Oak Executive Chair, Black",300.980,1,0.00,87.2842


DATA PREPROCESSING

In [46]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [47]:
df['State'].unique()

array(['California', 'Oregon', 'Arizona', 'Nevada', 'Washington',
       'Colorado', 'Utah', 'New Mexico', 'Idaho', 'Montana', 'Wyoming',
       'Texas', 'Illinois', 'Michigan', 'Indiana', 'South Dakota',
       'Wisconsin', 'Missouri', 'Minnesota', 'Iowa', 'Oklahoma',
       'Nebraska', 'Kansas', 'North Dakota', 'Pennsylvania', 'Delaware',
       'Ohio', 'New York', 'New Jersey', 'Massachusetts', 'Maryland',
       'Connecticut', 'New Hampshire', 'Maine', 'Rhode Island',
       'District of Columbia', 'Vermont', 'West Virginia', 'Georgia',
       'Kentucky', 'Virginia', 'Louisiana', 'South Carolina', 'Arkansas',
       'Tennessee', 'Florida', 'North Carolina', 'Mississippi', 'Alabama'],
      dtype=object)

In [48]:
df.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [49]:
df.describe(exclude='number')

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name
count,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994
unique,5009,1237,1334,4,793,793,3,1,531,49,4,1862,3,17,1850
top,CA-2014-100111,2015-09-06,2014-12-16,Standard Class,WB-21850,William Brown,Consumer,United States,New York City,California,West,OFF-PA-10001970,Office Supplies,Binders,Staple envelope
freq,14,38,35,5968,37,37,5191,9994,915,2001,3203,19,6026,1523,48


In [50]:
data_plus = df[df['Quantity'] >= 0]
data_plus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [51]:
top_10 = data_plus.groupby('State').nunique().sort_values('Order ID', ascending=False).head(10)
top_10

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
State,,,,,,,,,,,,,,,,,,,,
California,2001,1021,659,688,4,577,577,3,1,81,89,1,1204,3,17,1197,1542,14,3,1694
New York,1128,562,428,451,4,415,415,3,1,18,21,1,828,3,17,820,963,14,4,1026
Texas,985,487,391,406,4,370,370,3,1,54,59,1,773,3,17,763,859,13,6,893
Pennsylvania,587,288,254,262,4,257,257,3,1,8,11,1,496,3,17,494,525,14,5,549
Illinois,492,276,244,244,4,237,237,3,1,33,35,1,435,3,17,430,461,12,5,470
Washington,506,256,219,232,4,224,224,3,1,17,19,1,447,3,17,444,461,14,2,480
Ohio,469,236,201,211,4,202,202,3,1,24,24,1,411,3,17,410,441,12,5,451
Florida,383,200,186,180,4,181,181,3,1,36,38,1,345,3,17,341,357,11,4,363
North Carolina,249,136,131,128,4,122,122,3,1,19,19,1,229,3,17,227,236,14,4,242


In [52]:
top_10_transaction = pd.DataFrame(data_plus.groupby('State').nunique().sort_values('Order ID', ascending=False).head(10)['Order ID'])
top_10_transaction

,Order ID
State,
California,1021
New York,562
Texas,487
Pennsylvania,288
Illinois,276
Washington,256
Ohio,236
Florida,200
North Carolina,136


PLOTTING THE COUNTRY FROM TOP 10 BUYER

In [54]:
#total booking per segements
segments = top_10_transaction

#pie plot
fig = px.pie(segments,
            values=top_10_transaction['Order ID'],
            names = top_10_transaction.index,
            title= "State Performance by Number of Order",
            template= 'seaborn'
            )
fig.update_traces(rotation=-90, textinfo='percent+label')
fig.show()

from pie chart above we know that california held 28.5% of the transaction to buy items

CREATE GROUP OF ORDER IN CALIFORNIA

In [55]:
basked_plus = (data_plus[data_plus['State'] == 'California'].groupby(['Order ID', 'Sub-Category'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('Order ID'))
basked_plus

Sub-Category,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
Order ID,,,,,,,,,,,,,,,,,
CA-2011-100090,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
CA-2011-100860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2011-100867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
CA-2011-101462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2011-101931,0.0,0.0,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US-2014-160143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
US-2014-163300,0.0,0.0,0.0,7.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
US-2014-163657,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


ENCODING THE CALIFORNIA TRANSACTION DATA

In [56]:
def encode_units(x):
    if x <= 0:
        return 0
    if x>= 0:
        return 1

basked_encode_plus = basked_plus.applymap(encode_units)
basked_encode_plus

Sub-Category,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
Order ID,,,,,,,,,,,,,,,,,
CA-2011-100090,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
CA-2011-100860,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
CA-2011-100867,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
CA-2011-101462,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
CA-2011-101931,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US-2014-160143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
US-2014-163300,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0
US-2014-163657,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [57]:
basked_filter_plus = basked_encode_plus[(basked_encode_plus > 0).sum(axis=1) >= 2]
basked_filter_plus

Sub-Category,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
Order ID,,,,,,,,,,,,,,,,,
CA-2011-100090,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
CA-2011-101931,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0
CA-2011-102330,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
CA-2011-102652,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
CA-2011-103310,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US-2014-155999,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
US-2014-163300,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0
US-2014-163657,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


CALCULATE THE MARKET BASKET PREDICTION USING APRIORI


In [58]:
frequent_itemsets_plus = apriori(basked_filter_plus, min_support=0.03,
                                use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemsets_plus['length'] = frequent_itemsets_plus['itemsets'].apply(lambda x: len(x))
frequent_itemsets_plus

,support,itemsets,length
0,0.349680,(Binders),1
1,0.347548,(Paper),1
2,0.275053,(Phones),1
3,0.240938,(Furnishings),1
4,0.223881,(Storage),1
5,0.208955,(Art),1
6,0.196162,(Accessories),1
7,0.172708,(Chairs),1
8,0.155650,(Appliances),1
9,0.123667,(Labels),1


CREATE LENGTH OF ITEMS AND CALCULATE THE SUPPORT

In [59]:
frequent_itemsets_plus[ (frequent_itemsets_plus['length']  == 2) &
                        (frequent_itemsets_plus['support'] >= 0.03) ]

,support,itemsets,length
10,0.108742,"(Paper, Binders)",2
13,0.076759,"(Phones, Paper)",2
14,0.074627,"(Paper, Storage)",2
15,0.072495,"(Appliances, Binders)",2
16,0.072495,"(Phones, Binders)",2
19,0.070362,"(Furnishings, Binders)",2
20,0.068230,"(Art, Binders)",2
21,0.068230,"(Chairs, Paper)",2
22,0.066098,"(Art, Paper)",2
24,0.063966,"(Binders, Storage)",2


APPLIED ASSOCIATION RULES TO THE ITEM

In [60]:
association_rules(frequent_itemsets_plus, metric='lift',
                    min_threshold=1).sort_values('lift', ascending=False).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Binders),(Appliances),0.349680,0.155650,0.072495,0.207317,1.331941,0.018067,1.065180
1,(Appliances),(Binders),0.155650,0.349680,0.072495,0.465753,1.331941,0.018067,1.217265
2,(Appliances),(Furnishings),0.155650,0.240938,0.049041,0.315068,1.307674,0.011538,1.108230
3,(Furnishings),(Appliances),0.240938,0.155650,0.049041,0.203540,1.307674,0.011538,1.060128
4,(Chairs),(Appliances),0.172708,0.155650,0.034115,0.197531,1.269068,0.007233,1.052190
5,(Appliances),(Chairs),0.155650,0.172708,0.034115,0.219178,1.269068,0.007233,1.059514
6,(Paper),(Chairs),0.347548,0.172708,0.068230,0.196319,1.136711,0.008206,1.029379
7,(Chairs),(Paper),0.172708,0.347548,0.068230,0.395062,1.136711,0.008206,1.078543
8,(Phones),(Art),0.275053,0.208955,0.063966,0.232558,1.112957,0.006492,1.030755
9,(Art),(Phones),0.208955,0.275053,0.063966,0.306122,1.112957,0.006492,1.044776


FROM HERE WE KNOW THAT BLINDERS APPLIANCE WILL BE GOOD TO BE SOLD TOGETHER. BECAUSE THE "LIFT" VALUE US MORE THAN 1 AND THE BIGGER VALUE OF ALL THE DATA